<h1>
<center>
Data Science Part 3: Tune The Forest
</center>
</h1>
<div class=h1_cell>
By Fahad Alarefi. The purpose of this three-part project is to use machine learning into making use of a publicaly available table called IMDB 5000 titles.
The main idea is to come up with a function which can predictate, with high accuracy, whether a provided movie data will have 8 stars or more by the user community. The technique of decision trees will be used.
The dataset is accessed from here:
https://data.world/data-society/imdb-5000-movie-dataset

</div>
<br />
<div class=h1_cell>
In this part, we will continue from the last part (Part2: model the data), and tune the result to get better performance.<br />
Tuning the result means to use the some of trained models and try to get a better performance in accuracy and/or efficency. <br/>
We will use Random Forest model, and see if we are able to get a better model. <br/>
</div>

<h1>
Read the dataset from the last module
</h1>
<div class=h1_cell>
First step,
get the dataset from the last module we worked on.
</div>

In [1]:
import pandas as pd
import os
import dill
import pickle

if os.path.isfile('./part2_movie_table.csv'):
    file_name = 'part2_movie_table.csv'
else:
    file_name = 'https://github.com/createdbyfahad/datascience/files/2593722/part2_movie_table.txt'

movie_table = pd.read_csv(file_name)

with open ('rf', 'rb') as fp:
    forest = dill.load(fp)

target_column = "is_good2"

Also, in order not to rewrite the previous functions in last module, I have created a python file
<br/>that has all needed funcitons.
<br />So, here we will import it.

In [2]:
from functions2 import *

%who function

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_centroid_labels	 compute_mean	 compute_prediction	 compute_training	 
euclidean_distance	 execute_dt	 execute_rf	 f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 
informedness	 initialize_centroids	 k_fold_random	 k_means	 kmeans_fill	 list_oob	 oob_testing	 phase_1	 phase_2	 
predictor_case	 probabilities	 row_to_vect	 test_dt	 test_oob	 test_rf	 total_oob	 tree_predictor	 vote_taker	 
vote_taker_oob	 


In [3]:
movie_table.head()

,duration,gross,genres,country,content_rating,budget,title_year,imdb_score,Action,Adventure,...,forest_112,forest_112_type,forest_132,forest_132_type,forest_133,forest_133_type,forest_134,forest_134_type,tree_0,tree_0_type
0,178.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,USA,PG-13,237000000.0,2009.0,7.9,1,1,...,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
1,169.0,309404152.0,Action|Adventure|Fantasy,USA,PG-13,300000000.0,2007.0,7.1,1,1,...,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
2,148.0,200074175.0,Action|Adventure|Thriller,UK,PG-13,245000000.0,2015.0,6.8,1,1,...,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
3,164.0,448130642.0,Action|Thriller,USA,PG-13,250000000.0,2012.0,8.5,1,0,...,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
4,132.0,73058679.0,Action|Adventure|Sci-Fi,USA,PG-13,263700000.0,2012.0,6.6,1,1,...,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [4]:
test_oob(movie_table, forest, target_column)

(0.6674157303370787, 0.685374149659864, 0.33256428708034713)

<h1>
Inspecting the testing the forest
</h1>
<div class=h1_cell>
Our goal of this section of part 3 is to produce better results by exploring the trees behaviour. <br />
First, we will create a matrix with all rows of the table and each tree's vote. <br />
So, if I have a forest of 5 trees and data set of 100 rows then the matrix will be of 100 rows and 5 columns. 
</div>

In [5]:
# return a the trees prediction for a row in the paramater
def trees_row_prediciton(row, forest):
    result = []
    for tree in forest:
        prediction = tree_predictor(row, tree)
        result.append(prediction)
        
    return result
def votes_per_row(table, forest):
    matrices = [[] for i in range(len(table))]
    
    for i in range(len(table)):
        matrices[i] = trees_row_prediciton(table.iloc[i], forest);
    
    return matrices


In [6]:
all_votes = votes_per_row(movie_table, forest)
all_votes

[[1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,

Next, we need to find all rows that have the same prediction for all trees.

In [7]:
def unanimous_rows(all_votes):    
    tree_amount = len(all_votes[0])
    rows = []
    for row in range(len(all_votes)):
        init_val = all_votes[row][0];
        stop = 0
        for col in range(tree_amount):
            if all_votes[row][col] != init_val:
                stop = 1
                break
        if not stop:
            rows.append(row)
    
    return rows

In [8]:
unan_rows = unanimous_rows(all_votes)
len(unan_rows)

307

Looks like we have around 300 rows that have the same prediction for all trees.<br />
So, let's try to delete those rows and see if the results will be better or not.

In [9]:
movie_table2 = movie_table.drop(movie_table.index[unan_rows])

In [10]:
#movie_table = movie_table.reset_index()
def delete_from_list(forest, ulist):
    for tree in forest:
        tree['oob'] = [x for x in tree['oob'] if x not in ulist]
    return forest

In [11]:
forest2 = delete_from_list(forest, unan_rows)

In [12]:
test_oob(movie_table2, forest2, target_column)

(0.651218923485397, 0.6603995299647474, 0.30198025344445156)

Looks like that didn't produce better resutls, so we will try a similar procedure.<br />
We will look for trees that alway vote one vote (never change their vote)

In [13]:
def never_change(all_votes):
    tree_amount = len(all_votes[0])
    trees = []
    for col in range(tree_amount):
        init_val = all_votes[0][col];
        stop = 0
        for row in range(len(all_votes)):
            if all_votes[row][col] != init_val:
                stop = 1
                break
        if not stop:
            trees.append(col)
    
    return trees


In [14]:
nc = never_change(all_votes)
len(nc)

0

There are non such trees, so we will move on with additional tests.

<h1>
Inspecting the testing the forest
</h1>
<div class=h1_cell>
In this section, we will try two ways: delete most incorrect tree and delete the most correlated tree
</div>

In [15]:
def tree_accuracy(table, tree, target):
    # shouold return the accuracy of that tree on this table
    table['tree'] = table.apply(lambda row: tree_predictor(row, tree), axis=1)
    table['tree_type'] = table.apply(lambda row: predictor_case(row, pred='tree', target=target), axis=1)
    tree_types = table['tree_type'].value_counts()
    return accuracy(tree_types)

def delete_most_incorrect_tree(table, forest, target):
    # go over all trees in the forest 
    perf = []
    for tree in forest:
        perf.append(tree_accuracy(table, tree, target))
    #delete the lowest accuracy
    lowest = perf[0]
    lowest_id = 0
    for accur in range(len(perf)):
        if perf[accur] < lowest:
            lowest = perf[accur]
            lowest_id = accur
    del forest[lowest_id]
    return forest
        

In [16]:
new_forest = delete_most_incorrect_tree(movie_table, forest, target_column)

In [17]:
len(new_forest)

12

Now, retest the forest to see if it have got better or not.

In [18]:
test_oob(movie_table, new_forest, target_column)

(0.6447941888619855, 0.647779111644658, 0.2901300934247797)

That didn't enhance our forest model, so we will try another test which is to find trees that are highly correlated (always vote together)


In [19]:
def find_correlated_trees(table, tree_votes):
    # should retururn two most correlated trees
    votes = pd.DataFrame(tree_votes)
    corr = votes.corr()
    corr_val = corr[0][1]
    corr_ids = (0,1)
    for tree1 in range(len(corr)):
        for tree2 in range(len(corr)):
            if tree1 != tree2 and corr[tree1][tree2] > corr_val:
                corr_ids = (tree1, tree2)
                corr_val = corr[tree1][tree2]
                
    return corr_ids

In [20]:
correlated_trees = find_correlated_trees(movie_table, all_votes);
print(correlated_trees)

(5, 7)


Looks like the most correlated trees are number 10 and 11 <br />
So we will delete the one that has lower accuracy

In [21]:
forest3 = forest

tree1_accuracy = tree_accuracy(movie_table, forest[correlated_trees[0]], target_column)
tree2_accuracy = tree_accuracy(movie_table, forest[correlated_trees[1]], target_column)
if tree1_accuracy < tree2_accuracy:
    # delete tree 1
    del forest3[correlated_trees[0]]
else:
    del forest3[correlated_trees[1]]

Revaluate the forest by oob test

In [22]:
test_oob(movie_table, forest3, target_column)

(0.6430305973773677, 0.6426835196888673, 0.28703142437924534)

Similar or slightly better results than our last test, but still, not better than the initial result

<h1>
Final remarks
</h1>
<div class=h1_cell>
As my main goal is to enhance the forest model from last part, I wasn't able to come up with better results.<br />
As seen above, I tried four techniques to enhance the model: remove rows that have similar votes by all trees, delete trees that always predict the same result, <br />delete most incorrect tree, and delete most correlated tree. All four methods have resulted in better accuracy.
</div>

<h1>
Automated function
</h1>
<div class=h1_cell>
I have created a function that automates all what we did in this part. The function recieves a information about a dataset, and does all 4 tests we just did.< br />
Also, the function will try three sizes of forests and use the most accurate/efficent one.<br />
From last part, I found that a depth-max of 4 might be the best value, so it will be used as default.
</div> 

In [23]:
def model_training(table, column_choices, target, hypers):
    # this function will try to automate all what we did earlier.
    # first create check hyperparameter
    tree_n = 13 if 'total-trees' not in hypers else hypers['total-trees']
    k = hypers['max-depth'] if 'max-depth' in hypers else min(4, len(column_choices))
    
    forest_trials = [5, 11, 13]
    
    if 'total-trees' not in hypers:
        print("no trees number was given, so will try "+ str(len(forest_trials)) +" forest sizes")
        forest_perf = -float("inf")
        # try three forests and get the best one
        for n in forest_trials:
            # second build the forest and oob test it
            print("building a forest with " + str(n) + " trees and max-depth of "+ str(k))
            hypers = {"total-trees": n, 'max-depth': k}
            (table, test_forest) = execute_rf((n+1), table, column_choices, target, hypers);
            print("tree built. checking results using oob testing")
            test_results = test_oob(table, test_forest, target)
            print("accuracy: "+ str(test_results[0]) + " f1: "+ str(test_results[1]) + " informdness: " + str(test_results[2]))
            if sum(test_results) >= forest_perf:
                forest = test_forest
                results = test_results
                forest_perf = sum(results)
        print("forest with " + str(len(forest)) + " has the best results, so will continue with it.")
    all_votes = votes_per_row(table, forest)
    
    #start doing our tests
    print("doing test 1 out of 4: delete rows with similar tree predictions")
    unan_rows = unanimous_rows(all_votes)
    if len(unan_rows) > 0:
        test_table = table.drop(movie_table.index[unan_rows]) 
        test_forest = delete_from_list(forest, unan_rows)
        test_results = test_oob(test_table, test_forest, target)
        print("test 1 accuracy: "+ str(test_results[0]) + " f1: "+ str(test_results[1]) + " informdness: " + str(test_results[2]))
        # check if results are better
        if sum(test_results)/len(test_results) >= sum(results)/len(results):
            print("test 1 resulted better or similar outcome so will continue with it.")
            table = test_table.copy()
            forest = test_forest
            results = test_results
            all_votes = votes_per_row(table, forest)
        else:
            print("test 1 resulted in worse outcome so will stick with pre test data.")
    else:
        print("test 1 results affects nothing")
    
    
    #test 2
    print("doing test 2 out of 4: delete trees with similar predictions for all rows")
    nc = never_change(all_votes)
    if len(nc) > 0:
        test_table = table.copy() 
        test_forest = forest
        test_results = test_oob(test_table, test_forest, target)
        print("test 2 accuracy: "+ str(test_results[0]) + " f1: "+ str(test_results[1]) + " informdness: " + str(test_results[2]))
        # check if results are better
        if sum(test_results)/len(test_results) >= sum(results)/len(results):
            print("test 2 resulted better or similar outcome so will continue with it.")
            table = test_table.copy()
            forest = test_forest
            results = test_results
            all_votes = votes_per_row(table, forest)
        else:
            print("test 2 resulted in worse outcome so will stick with pre test data.")
    else:
        print("test 2 results affects nothing")
        
        
    #test 3
    print("doing test 3 out of 4: delete the most incorrect tree")
    test_table = table.copy() 
    test_forest = delete_most_incorrect_tree(movie_table, forest, target_column)
    test_results = test_oob(test_table, test_forest, target)
    print("test 3 accuracy: "+ str(test_results[0]) + " f1: "+ str(test_results[1]) + " informdness: " + str(test_results[2]))
    # check if results are better
    if sum(test_results)/len(test_results) >= sum(results)/len(results):
        print("test 3 resulted better or similar outcome so will continue with it.")
        table = test_table.copy()
        forest = test_forest
        results = test_results
        all_votes = votes_per_row(table, forest)
    else:
        print("test 3 resulted in worse outcome so will stick with pre test data.")
    
    #test 4
    print("doing test 4 out of 4: remove the most correlated tree")
    test_table = table.copy()
    correlated_trees = find_correlated_trees(table, all_votes);
    if(len(correlated_trees) == 2):
        test_forest = forest
#         print(len(forest))
#         print(correlated_trees)
        tree1_accuracy = tree_accuracy(movie_table, forest[correlated_trees[0]], target_column)
        tree2_accuracy = tree_accuracy(movie_table, forest[correlated_trees[1]], target_column)
        
        if tree1_accuracy < tree2_accuracy:
            del test_forest[correlated_trees[0]]
        else:
            del test_forest[correlated_trees[1]]
        test_results = test_oob(test_table, test_forest, target)
        print("test 4 accuracy: "+ str(test_results[0]) + " f1: "+ str(test_results[1]) + " informdness: " + str(test_results[2]))
        # check if results are better
        if sum(test_results)/len(test_results) >= sum(results)/len(results):
            print("test 4 resulted better or similar outcome so will continue with it.")
            table = test_table.copy()
            forest = test_forest
            results = test_results
        else:
            print("test 4 resulted in worse outcome so will stick with pre test data.")
    else:
        print("test 4 results affects nothing")
    
    
    return (table, forest)
    

In [24]:
(table, trained_model) = model_training(movie_table, splitter_columns, target_column, {})

no trees number was given, so will try 3 forest sizes
building a forest with 5 trees and max-depth of 4
tree built. checking results using oob testing
accuracy: 0.6484667165295438 f1: 0.6356589147286822 informdness: 0.302572518844362
building a forest with 11 trees and max-depth of 4
tree built. checking results using oob testing
accuracy: 0.64659449706811 f1: 0.6881592039800994 informdness: 0.2828734583370818
building a forest with 13 trees and max-depth of 4
tree built. checking results using oob testing
accuracy: 0.6419586702605571 f1: 0.6258215962441315 informdness: 0.292120541930303
forest with 11 has the best results, so will continue with it.
doing test 1 out of 4: delete rows with similar tree predictions
test 1 accuracy: 0.6221267996968931 f1: 0.6443176414645744 informdness: 0.24606778856339195
test 1 resulted in worse outcome so will stick with pre test data.
doing test 2 out of 4: delete trees with similar predictions for all rows
test 2 results affects nothing
doing test 3 

Then the trained model can be used in predicting a movie success (with low accuracy) or for working on it and increase it's effiency